# Data pre-processing

### Importing Important  librararies

In [1]:
import pandas as pd
import re
import warnings
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mosta\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# !{sys.executable} -m pip install TextBlob
# !python -m textblob.download_corpora

In [3]:
# pip install textblob

### Read The Dataset

In [4]:
df=pd.read_csv('data.csv')
df.head()

,id,dialect,text
0,1175358310087892992,IQ,@Nw8ieJUwaCAAreT لكن بالنهاية .. ينتفض .. يغير .
1,1175416117793349632,IQ,@7zNqXP0yrODdRjK يعني هذا محسوب على البشر .. ح...
2,1175450108898565888,IQ,@KanaanRema مبين من كلامه خليجي
3,1175471073770573824,IQ,@HAIDER76128900 يسلملي مرورك وروحك الحلوه💐
4,1175496913145217024,IQ,@hmo2406 وين هل الغيبه اخ محمد 🌸🌺


### Removing hashtags and new lines from tweets and Replace @username with empty string then Remove new lines with white space

In [5]:
ar_stops = set(stopwords.words('arabic'))
def remove_stop_words(text):
    zen = TextBlob(text)
    words = zen.words
    return " ".join([w for w in words if not w in ar_stops])

### remove repeated letters

In [6]:
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1\1', text)

In [7]:
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

In [8]:
import emoji
def give_emoji_free_text(text):
    regrex_pattern = re.compile(pattern = "["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                       "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

In [9]:
#start processing the tweet
def processPost(tweet):

    #Replace @username with empty string
    tweet = re.sub('@[^\s]+', ' ', tweet)
    
    #Convert www.* or https?://* to " "
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',tweet) 

     # remove underscore
    tweet = re.sub('[_]', ' ',  tweet) 

    #tweet= remove_punctuations(tweet)

    # remove repeated letters
    tweet=remove_repeating_char(tweet)
    tweet=give_emoji_free_text(tweet)

    #Remove new lines with white space
    tweet = tweet.replace('\n',' ')
    tweet = tweet.replace('\t',' ')
    
    
    # remove Arabic stop words from the tweets
    tweet = remove_stop_words(tweet)    
   
    # normalize the tweet
    tweet= normalize_arabic(tweet)
        
    return tweet

In [10]:
df["clean text"] = df['text'].apply(lambda x: processPost(x))

In [11]:
df.head(10)

,id,dialect,text,clean text
0,1175358310087892992,IQ,@Nw8ieJUwaCAAreT لكن بالنهاية .. ينتفض .. يغير .,بالنهايه ينتفض يغير
1,1175416117793349632,IQ,@7zNqXP0yrODdRjK يعني هذا محسوب على البشر .. ح...,يعني محسوب البشر حيونه ووحشيه وتطلبون الغرب يح...
2,1175450108898565888,IQ,@KanaanRema مبين من كلامه خليجي,مبين كلامه خليجي
3,1175471073770573824,IQ,@HAIDER76128900 يسلملي مرورك وروحك الحلوه💐,يسلملي مرورك وروحك الحلوه
4,1175496913145217024,IQ,@hmo2406 وين هل الغيبه اخ محمد 🌸🌺,وين الغيبه اخ محمد
5,1175668034146643968,IQ,@Badi9595 @KanaanRema يااخي الإرهابي اذا كان ع...,يااخي الارهابي اذا عراقي سعودي فلسطيني وين الم...
6,1175670153884983296,IQ,@SarahNadhum90 @nUBNTdfVgACYQxV مطلبي يقدم است...,مطلبي يقدم استقالته وفوكاها اعتذار
7,1175671762580856832,IQ,@KanaanRema @Badi9595 خلص والله لعيونكم انا ما...,خلص والله لعيونكم انا ماعندي شيء معه بالعكس مت...
8,1175715664398561280,IQ,@SalahAlarbawi يمكن سؤال فات الكثير اللي يصور ...,يمكن سؤال فات الكثير اللي يصور شنو موقفه وكانه...
9,1176019816072777728,IQ,@Eng_alow91 @cb4LwpWrS1hT5lb @EdyCohen اولا ان...,اولا اني ردت رجل جنوبي واللي ذكر حجابها ثانياً...


### Saving The new dataset 

In [12]:
df.to_csv("clean_data.csv",index=False)